In [3]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip, AudioFileClip
from PIL import Image
import cv2
import os

In [4]:
# 초당 프레임 30 ; 1분이면 너무 많아요 - 10초만 할게요
input_file = "video/video.mp4"  # 입력 MP4 파일 경로
output_file = "video/video_trimmed.mp4"  # 출력 MP4 파일 경로

start_time = 0  # 자를 시작 시간 (초)
end_time = 11  # 자를 종료 시간 (초)

ffmpeg_extract_subclip(input_file, start_time, end_time, targetname=output_file)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [5]:
video_path = "video/video_trimmed.mp4"

In [6]:
# 영상 가로세로 크기 확인
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("가로 크기:", frame_width)
print("세로 크기:", frame_height)

cap.release()

가로 크기: 720
세로 크기: 1280


In [7]:
def resize_and_save_frames(video_path, output_dir, top_crop, bottom_crop):

  # 비디오 파일 열기
  cap = cv2.VideoCapture(video_path)

  # 프레임 수, 프레임 크기 가져오기
  num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  # 초당 프레임 수 출력
  total_duration = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS))
  fps = num_frames / total_duration
  print("초당 프레임 수:", fps)

  # 저장할 이미지 디렉토리 생성
  os.makedirs(output_dir, exist_ok=True)

  # 비디오 프레임 읽기
  for frame_idx in range(num_frames):
    ret, frame = cap.read()

    if ret:
      # 프레임 크기 변경 (필요할 때)
      cropped_frame = frame[top_crop:-bottom_crop, :]

      # 이미지 파일 경로
      image_path = os.path.join(output_dir, f"frame_{frame_idx:03}.png")

      # 이미지 저장
      cv2.imwrite(image_path, cropped_frame)
      # 크기 변경하지 않았을 때 cropped_frame = frame

  cap.release()

  print("프레임 이미지 저장 완료!")
  print("이미지 개수:", len(os.listdir(output_dir)))

In [8]:
# 음성 추출
def extract_audio(video_path, output_audio):
  video = VideoFileClip(video_path)
  audio = video.audio
  audio.write_audiofile(output_audio, codec='libmp3lame')

In [10]:
# 이미지 크기 줄일 때 사용
top_crop = 280
bottom_crop = 280

# 오디오 생성 경로
output_audio = 'audio/audio.wav'

# 프레임 생성 경로
output_dir = 'file'

# 프레임 및 음성 추출
resize_and_save_frames(video_path, output_dir, top_crop, bottom_crop)
extract_audio(video_path, output_audio)

초당 프레임 수: 30.09090909090909
프레임 이미지 저장 완료!
이미지 개수: 330


chunk:   9%|▉         | 22/244 [00:44<07:28,  2.02s/it, now=None]

MoviePy - Writing audio in audio/audio.wav


chunk:   9%|▉         | 22/244 [00:44<07:30,  2.03s/it, now=None]

MoviePy - Done.


이후에는 Roop Extension 작업을 거친 뒤 실행합니다.

In [9]:
# 중간 보고 - GIF 만들기
folder_path = "Roop_image"  # 프레임 이미지들이 있는 폴더 경로
output_file = "gif/output.gif"  # 출력 GIF 파일 경로

frame_images = []  # 프레임 이미지 파일들을 저장할 리스트

# 폴더 내의 이미지 파일들을 순서대로 읽어와서 리스트에 추가
for filename in sorted(os.listdir(folder_path)):
  if filename.startswith("frame_") and filename.endswith(".png"):
    image_path = os.path.join(folder_path, filename)
    frame_images.append(Image.open(image_path))

# X ~ Y번째의 프레임으로만 gif 저장할 때 쓰는 코드
frame_images = frame_images[:30] # [X:Y]

# GIF로 저장 / duration - 초당 프레임
frame_images[0].save(output_file, format="GIF", append_images=frame_images[1:], save_all=True, duration=50, loop=0)

In [19]:
# 비디오만 저장, (오디오 변환 시 필요)

# 이미지 파일 경로
image_folder = 'Roop_image'
# 영상 프레임 사이즈
frame_size = (720, 720)

# 비디오 저장을 위한 설정
video_name = 'video/Roop_video/Non_wav_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(video_name, fourcc, 30.0, frame_size)

# 이미지 파일 목록 가져오기
image_files = sorted(os.listdir(image_folder))

# 각 이미지를 영상에 추가
for image_file in image_files:
  image_path = os.path.join(image_folder, image_file)
  frame = cv2.imread(image_path)
  frame = cv2.resize(frame, frame_size)
  video_writer.write(frame)

# 작업 완료 후 비디오 릴리즈
video_writer.release()

In [20]:
# 비디오 파일 경로
video_path = video_name
# 오디오 파일 경로
audio_path = 'audio/audio.wav'

# 비디오와 오디오 합성
video = VideoFileClip(video_path)
audio = AudioFileClip(audio_path)
video = video.set_audio(audio)

# 합성된 비디오 저장
output_path = 'video/Roop_video/final_video.mp4'
video.write_videofile(output_path)

Moviepy - Building video /content/drive/MyDrive/AI_code/Real/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/AI_code/Real/final_video.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/AI_code/Real/final_video.mp4
